<a href="https://colab.research.google.com/github/amasl0/myKNN/blob/main/VK_friends_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### import libraries

In [ ]:
import pandas as pd
import numpy as np
import json
import requests
from tqdm import tqdm
import os 
import networkx as nx
import csv
import ast
from google.colab import drive

drive.mount('/content/drive')

PATH_STORAGE = '/content/drive/MyDrive/parts'
PATH_EXCEPTIONS = '/content/drive/MyDrive/Exceptions'
PATH_ALL_PATH = '/content/drive/MyDrive/All_pathes'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Берём друзей студентов из ВК

In [ ]:
lst_group_students = pd.read_csv( '/'.join((PATH_STORAGE ,'firts_level_friends.csv'))).main_id.unique().tolist()

In [ ]:
with open (os.path.join(PATH_STORAGE ,'token.txt'), 'r') as f:
    token = str(f.readline())
    
def getFriend (token, user_id, level_friend):
    list_data = []

    ## могут быть ошибки из-за некорректного json
    response = requests.get("https://api.vk.com/method/friends.get",
                        params={
                            "user_id": user_id,
                            "order": "name",
                            "count": 10000,
                            "offset": 0,
                             "fields": "nickname",
                            "access_token": token,
                            "v": 5.126,
                    }).json()["response"]

    if not "error" in response.keys():
        if response['items'] != None:
            for friend in response['items']:
                if not "deactivated" in friend and friend["is_closed"] != True:
                    list_data.append([ user_id, level_friend, friend['id'], friend['first_name'], friend['last_name'] ])
    return list_data


In [ ]:
import time

al_dic = []

## итерируемся по группе
for student_id in lst_group_students:
    res = getFriend(token, student_id, 1)
    if type(res) == list and res != None:
        al_dic.extend(res)
    time.sleep(0.05)

In [ ]:
df_friends = pd.DataFrame(al_dic,columns=['main_id','level_friend','id_friend','first_name','last_name'])

In [ ]:
df_friends

,main_id,level_friend,id_friend,first_name,last_name
0,181948702,1,442387982,ピンクの雨,カップ・カップ
1,181948702,1,581054682,叫张,志
2,181948702,1,143779216,Egor,Ruzaikin
3,181948702,1,170844725,Elena,Bannikova
4,181948702,1,218562377,Geor,Bulop
...,...,...,...,...,...
3026,312616596,1,385947754,Цзо,Женя
3027,312616596,1,347822077,Шараф,Нарзуллоев
3028,312616596,1,690298174,Шахром,Шарипов
3029,312616596,1,474174041,Юрий,Нано


In [ ]:
## удаляем существующие части в parts, если они существуют

if len(os.listdir(PATH_STORAGE)) != 0:
    for filename in os.listdir(PATH_STORAGE):
        if 'part' in filename:
            os.remove(os.path.join(PATH_STORAGE, filename) ) 

29


In [ ]:
al_dic = []
df_second_friends = pd.DataFrame()
counter = 0

for tup in tqdm(df_friends.itertuples(), total=df_friends.shape[0]):
    try:
        res = getFriend(token, tup.id_friend, 2)
    except:
        ## если обнаружились ошибки чтении json, запишем в файл для анализа
        error_response = requests.get("https://api.vk.com/method/friends.get",
                    params={
                            "user_id": tup.id_friend,
                            "order": "name",
                            "count": 10000,
                            "offset": 0,
                             "fields": "nickname",
                            "access_token": token,
                            "v": 5.126,
                    })
        with open( 
            os.path.join(PATH_EXCEPTIONS, f'exception-{counter}.txt')
            , 'w') as except_manager:
            except_manager.write(str(error_response.text))
        continue

    if type(res) == list and res != None:
        al_dic.extend(res)
    time.sleep(0.05)
    
    if len(al_dic) > 100000:
        pd.DataFrame(al_dic, 
                     columns=['main_id','level_friend','id_friend','first_name','last_name']
                ).to_csv(os.path.join(PATH_STORAGE, f'part-{counter}.csv'), index=False)
        counter += 1
        al_dic.clear()

## Сохранение остатка

pd.DataFrame(
    al_dic, columns=[
        'main_id','level_friend','id_friend','first_name','last_name']
    ).to_csv(os.path.join(PATH_STORAGE, f'part-{counter}.csv'), index=False)

100%|██████████| 3022/3022 [48:10<00:00,  1.05it/s]


### Объединяем все части в один dataframe

In [ ]:
for part in os.listdir(PATH_STORAGE):
    if part[-4:] == '.csv' and 'part' in part:
        df_friends = df_friends.append( pd.read_csv(  os.path.join( PATH_STORAGE, part )  ) )

In [ ]:
df_friends = df_friends.drop_duplicates()

In [ ]:
df_friends

,main_id,level_friend,id_friend,first_name,last_name
0,181948702,1,442387982,ピンクの雨,カップ・カップ
1,181948702,1,581054682,叫张,志
2,181948702,1,143779216,Egor,Ruzaikin
3,181948702,1,170844725,Elena,Bannikova
4,181948702,1,218562377,Geor,Bulop
...,...,...,...,...,...
40303,295707677,2,359769196,Шахром,Аминов
40304,295707677,2,312616596,Шохин,Гулямов
40305,295707677,2,261840457,Шохрух,Сидиков
40306,295707677,2,468207716,Эди,Назаров


### Делаем граф в networkx

In [ ]:
G = nx.Graph()

In [ ]:
### нужно узнать id уникальных пользователей
## добавляем вершины графа
G.add_nodes_from(set( df_friends['main_id'].unique()) | set(df_friends['id_friend'].unique()))
G.add_edges_from( list(df_friends[['main_id', 'id_friend']].itertuples(index=False, name=None)) )

### Степень центральности по посредничеству

In [ ]:
## удаляем существующие части в parts
# for filename in os.listdir(PATH_ALL_PATH):
#     if 'part' in filename:
#         os.remove(os.path.join(PATH_ALL_PATH, filename) ) 

In [ ]:
pathes = []

counter_parts = 0

for student_id in lst_group_students:
    for friends_id in tqdm(set(G.nodes) - set([student_id])):
        path = nx.shortest_path(G, source=student_id, target=friends_id)
        if len(pathes) > 100000:
            with open( os.path.join(PATH_ALL_PATH, f'part-{counter_parts}.txt'), "w") as fb:
                fb.write(str(pathes))
                counter_parts += 1
                pathes.clear()
        pathes.append(path)

with open( os.path.join(PATH_ALL_PATH, f'part-{counter_parts}.txt'), "w") as fb:
    fb.write(str(pathes))
    pathes.clear()

 18%|█▊        | 188024/1043062 [08:11<31:46, 448.43it/s]

In [ ]:
count_nodes = len(G.nodes)
dict_closeoness = {}

## анализируем каждый файл и подсчитываем элементы
for filename in tqdm(os.listdir(PATH_ALL_PATH)):
    if filename.split('.')[-1] != 'txt':
        continue
    with open(os.path.join(PATH_ALL_PATH, filename), "r") as reader:
        pathes = ast.literal_eval(reader.read())
        
    for path in pathes:
        ## if exist
        if path[0] in dict_closeoness:
            dict_closeoness[path[0]] += len(path[1:-1])
        else:
            dict_closeoness[path[0]] = len(path[1:-1])

100%|██████████| 174/174 [17:14<00:00,  5.95s/it]


In [ ]:
res_close = {}

for key in dict_closeoness:
    res_close[key] = count_nodes / dict_closeoness[key]

sorted_dict = {}

for w in sorted(res_close, key=res_close.get, reverse=True) :
    sorted_dict[w] = res_close[w]

df_friends.loc[(
            df_friends.id_friend ==
                    list(sorted_dict)[0]
                ), ['first_name', 'last_name']
         ].head(1)

,first_name,last_name
786,Михаил,Никифоров
